In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount("/content/drive", force_remount=True)

In [ ]:
import pandas as pd
import os
import re
from multiprocessing import Pool

input_dir = "/content/drive/MyDrive/Thesis/Original_dataset"
output_csv = "/content/drive/MyDrive/Thesis/output_tweets.csv"
num_workers = 4

def extract_views(v):
    m = re.search(r"'count':\s*'([^']*)'", str(v))
    return m.group(1) if m else None

def handle_file(path):
    df = pd.read_csv(path)
    df = df[(df['lang'] == 'en') & (df['retweetedTweet'] == 'FALSE')]
    df['viewCount'] = df['viewCount'].apply(extract_views)
    return df[['text', 'viewCount', 'likeCount', 'quoteCount', 'replyCount', 'retweetCount']]

def find_csvs(folder):
    paths = []
    for root, _, files in os.walk(folder):
        for f in files:
            if f.endswith('.csv'):
                paths.append(os.path.join(root, f))
    return paths

if __name__ == "__main__":
    files = find_csvs(input_dir)
    print(f"Total:{len(files)}")

    with Pool(num_workers) as p:
        results = p.map(handle_file, files)

    dfs_search = [r for r in results if r is not None and not r.empty]
    final = pd.concat(dfs_search, ignore_index=True)
    final.to_csv(output_csv, index=False)
    print(f"Rows saved: {len(final)}")